<a href="https://colab.research.google.com/github/zeekx/build_dl_framework/blob/dev/steps/step18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backward: vector, x and itself

x = Variable(np.array(1.0))
y = add(x, x)
y.backward()
print("y=x+x=2x, f'(1)=2, but x.grad=", x.grad)



In [74]:
import numpy as np
import heapq
import weakref

In [75]:
class Variable:
    def __init__(self, data):
        if data is not None:
          if not isinstance(data, np.ndarray):
            raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def reset_grad(self):
        self.grad = None

# a -> f(a) -> b
# Variable: b.creator -> f
# b.creator -> f -> f.input -> a

    def backward(self, retain_grad = False):
        def pop_func(alist):
            return heapq.heappop(alist)
        
        def append_func(alist, f):
            heapq.heappush(alist, f)

        
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        fs = [self.creator]            
        while fs:
            f = pop_func(fs)
            ygs = [o().grad for o in f.outputs]
            xgs = f.backward(*ygs)
            if not isinstance(xgs, tuple):
                xgs = (xgs,)
    
            for x, g in zip(f.inputs, xgs):
                if x.grad is None:
                    x.grad = g
                    if x.creator is not None: # append the function once when the var: x first appeared
                        append_func(fs, x.creator)
                else: # Var:x, is repeated
                    x.grad = x.grad + g
            
            # for the very begin of variables, their backward codes Do NOT reach here, because they don't have any creator
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None # y is a weakref of the output


In [76]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [77]:
class Config:
    enable_backprop = True # backward propagate

In [78]:
class Function:
    def __init__(self) -> None:
        self.generation = 0
    
    def __lt__(self, other):
            return -self.generation < -other.generation
    
    def __call__(self, *inputs): # input:[x0, x1, ...]@Variable -> [y0, y1, ...]@Variable
        self.inputs = inputs
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #unwrap
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in self.inputs])
            for output in outputs:
                output.set_creator(self)
            self.outputs = [weakref.ref(o) for o in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


In [79]:

class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx


In [80]:
class Add(Function):
    def forward(self, a, b):
        y = a + b
        return (y,)
    
    def backward(self, gy):
        return gy, gy

In [81]:
class Identical(Function):
    def forward(self, x):
        return x
    
    def backward(self, gy):
        return gy # ??? 1 or gy

In [82]:
def square(x):
  return Square()(x)

def exp(x):
  return Exp()(x)

def add(x0, x1):
  return Add()(x0, x1)

def identical(x):
  return Identical()(x)

In [83]:
SIZE = (100, 100, 100)

# Test Code
Config.enable_backprop = True
x = Variable(np.ones(SIZE))
y = square(square(square(x))) 
y.backward()

Config.enable_backprop =False
x = Variable(np.ones(SIZE))
y =square(square(square(x))) 
y.backward()

In [84]:
# context

import contextlib

@contextlib.contextmanager
def config_test():
    print('start')
    try:
        yield
    finally:
        print('Done')


with config_test():
    print('process...')

start
process...
Done


In [85]:
@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

In [86]:
with using_config('enable_backprop', False):
    x = Variable(np.ones(SIZE))
    y =square(x)

In [87]:
def no_grad():
    return using_config( 'enable_backprop' , False)

with no_grad():
    x =Variable(np.array(2.0))
    y = square(x)